In [2]:
import os

import dspy
from dotenv import load_dotenv

In [3]:
%load_ext dotenv
%dotenv

### 計算

In [8]:
lm = dspy.LM("openai/gpt-4o-mini", api_key=os.environ.get("API_KEY"))
dspy.configure(lm=lm)

math = dspy.ChainOfThought("question -> answer")
res = math(question="2の3乗は？")

In [9]:
res.answer

'8'

### 情報抽出

In [ ]:

# class ExtractInfo(dspy.Signature):
#     """Extract structured information from text."""

#     text: str = dspy.InputField()
#     title: str = dspy.OutputField()
#     headings: list[str] = dspy.OutputField()
#     entities: list[dict[str, str]] = dspy.OutputField(desc="日付の起点と終点")

# module = dspy.Predict(ExtractInfo)

# text = "2002/1/2から2004/5/5まで"
# response = module(text=text)

# print(response.title)
# print(response.headings)
# print(response.entities)

日付の範囲
['開始日', '終了日']
[{'開始日': '2002/1/2', '終了日': '2004/5/5'}]


In [22]:

from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal["positive", "negative", "neutral"] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)

In [23]:
import dspy

def metric(example, pred, trace=None) -> float:
  return float(example.sentiment == pred.sentiment)

class MyPromptModule(dspy.Signature):
    """Example: 文章から感情を予測する"""
    text = dspy.InputField()
    sentiment = dspy.OutputField()

classify = dspy.ChainOfThoughtWithHint(MyPromptModule)

# 最適化用の学習データ（few-shot）
train_data = [
    dspy.Example(text="今日はとても楽しかった！", sentiment="positive"),
    dspy.Example(text="全然面白くなかった。", sentiment="negative"),
]

# プロンプトの最適化
optimizer = dspy.BootstrapFewShotWithRandomSearch(metric=(lambda x, y, trace=None: x.sentiment == y.sentiment), num_threads=10)
optimized = optimizer.compile(classify, trainset=train_data)

# 最適化されたプロンプトを使用して推論
result = optimized(text="素晴らしい体験だった")
print(result.sentiment)

2025/08/03 05:23:50 ERROR dspy.utils.parallelizer: Error for Example({'text': '今日はとても楽しかった！', 'sentiment': 'positive'}) (input_keys=None): Inputs have not been set for this example. Use `example.with_inputs()` to set them.. Set `provide_traceback=True` for traceback.
2025/08/03 05:23:50 ERROR dspy.utils.parallelizer: Error for Example({'text': '全然面白くなかった。', 'sentiment': 'negative'}) (input_keys=None): Inputs have not been set for this example. Use `example.with_inputs()` to set them.. Set `provide_traceback=True` for traceback.


Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 16 candidate sets.
Average Metric: 0.00 / 0 (0%): 100%|██████████| 2/2 [00:00<00:00, 417.88it/s]

2025/08/03 05:23:50 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 2 (0.0%)
2025/08/03 05:23:50 ERROR dspy.utils.parallelizer: Error for Example({'text': '今日はとても楽しかった！', 'sentiment': 'positive'}) (input_keys=None): Inputs have not been set for this example. Use `example.with_inputs()` to set them.. Set `provide_traceback=True` for traceback.
2025/08/03 05:23:50 ERROR dspy.utils.parallelizer: Error for Example({'text': '全然面白くなかった。', 'sentiment': 'negative'}) (input_keys=None): Inputs have not been set for this example. Use `example.with_inputs()` to set them.. Set `provide_traceback=True` for traceback.



New best score: 0.0 for seed -3
Scores so far: [0.0]
Best score so far: 0.0
Average Metric: 0.00 / 0 (0%): 100%|██████████| 2/2 [00:00<00:00, 415.57it/s]

2025/08/03 05:23:50 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 2 (0.0%)


...

Average Metric: 0.00 / 0 (0%): 100%|██████████| 2/2 [00:00<00:00, 541.79it/s]

2025/08/03 05:23:50 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 2 (0.0%)



Scores so far: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Best score so far: 0.0
19 candidate programs found.


2025/08/03 05:23:55 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


RuntimeError: Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.BadRequestError: OpenAIException - Unrecognized request argument supplied: rationale_type